# Extracting data from MovieAPI DataBase
We realize that there is a limit but worked around to find that MovieAPI does a daily dump for us

In [38]:
# For data processing
import numpy as np
import pandas as pd

# For API usage
import requests as r

#our lifesaver, the progress bar
from tqdm import tqdm

In [8]:
#API Key 
API_KEY = 'b4c353d61e8791dac39e77e69a2c08d3' 

##This is to extract our the whole database from their daily dump
simple_extract = pd.read_json('datasets/movie_ids_03_25_2023.json', lines=True)
print(simple_extract.dtypes)
print(simple_extract.head(5))
 

adult                bool
id                  int64
original_title     object
popularity        float64
video                bool
dtype: object
   adult     id                      original_title  popularity  video
0  False   3924                             Blondie       3.502  False
1  False   6124                 Der Mann ohne Namen       0.600  False
2  False   8773                 L'Amour à vingt ans       2.722  False
3  False  25449  New World Disorder 9: Never Enough       1.097  False
4  False  31975      Sesame Street: Elmo Loves You!       0.600   True


In [22]:
cleanedSimple_data = simple_extract.drop(simple_extract[(simple_extract['adult'] == True) | (simple_extract['video'] == True)].index)
print(cleanedSimple_data.head(5))

#Verify adult videos and videos are false
print(cleanedSimple_data['adult'].unique())
print(cleanedSimple_data['video'].unique())

   adult     id                      original_title  popularity  video
0  False   3924                             Blondie       3.502  False
1  False   6124                 Der Mann ohne Namen       0.600  False
2  False   8773                 L'Amour à vingt ans       2.722  False
3  False  25449  New World Disorder 9: Never Enough       1.097  False
5  False      2                               Ariel       9.553  False
[False]
[False]


In [40]:
#We'll only need our ID column for this to create our actual dataset
movie_data = cleanedSimple_data.drop(columns=['adult','video'])
movie_data = movie_data.sort_values(by=['id'])
print(movie_data.head(5))

   id                    original_title  popularity
5   2                             Ariel       9.553
6   3              Varjoja paratiisissa       9.228
7   5                        Four Rooms      18.254
8   6                    Judgment Night      11.309
9   8  Life in Loops (A Megacities RMX)       2.516


In [39]:
#movie ID gets stats of movie
#credit get 

#Get Movie Stats through MovieID
cols = ['release_date','vote_average', 'popularity', 'budget','revenue']
for i in tqdm(range(len(movie_data))):
    i = movie_data[i]
    resp = r.get(f'https://api.themoviedb.org/3/movie/{i}?api_key={API_KEY}')
    rows = []
    try:
        budget = resp.json()['budget']
        if(budget == 0):
            movie_data['budget'] = np.nan
        else:
            movie_data['budget'] = budget
    except KeyError:
        movie_data['budget'] = np.nan
    try:
        revenue = resp.json()['revenue']
        if(revenue == 0):
            movie_data['revenue'] = np.nan
        else:
            movie_data['revenue'] = revenue
    except KeyError:
        movie_data['revenue'] = np.nan
    try:
        release = resp.json()['release_date'][0:4]
        movie_data['release_date'] = release
    except:
        rows.append(np.nan)
    
    try:
        remaining = resp.json()['popularity']
        if(remaining == 0):
            movie_data['popularity'] = np.nan
        else:
            movie_data['popularity'] = remaining
    except:
        movie_data['popularity'] = np.nan

    try:
        remaining = resp.json()['vote_average']
        if(remaining == 0):
            movie_data['vote_average'] = np.nan
        else:
            movie_data['vote_average'] = remaining
    except:
        movie_data['vote_average'] = np.nan

print(movie_data.head(5))


  0%|          | 0/751618 [00:00<?, ?it/s]


TypeError: 'int' object is not subscriptable